In [123]:
import configparser
import requests
from requests import Session
from psycopg2 import connect
from psycopg2.extras import execute_values
from time import sleep
CONFIG = configparser.ConfigParser()
CONFIG.read(r'C:\Users\nchan6\Documents\db.cfg')
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)

In [214]:
def get_url(mapserver, id):
    "This prints the base_url for the request"
    url = "https://insideto-gis.toronto.ca/arcgis/rest/services/{}/MapServer/{}/query".format(mapserver, id)
    return(url)

In [299]:
def check_status(max_number, record_max):
    "This gets the request status"
    try:
        r = requests.get(base_url, params = query, verify = False)
    except requests.exceptions.SSLError:
        sleep(30)
        status = 'error_raised'
    else:
        status = 'successful'
    return status

In [300]:
def get_layer(mapserver, id, output_table):
    
    """
    This function calls to the GCCview rest API and inserts the outputs to the output table in the postgres database.

    Parameters
    ----------
    mapserver : string
        The name of the mapserver that host the desire layer

    id : int
        The id of desire layer

    output_table: string
        Name of table that returned row will be inserted into
    """  
    base_url = get_url({}, {}).format(mapserver, id)
    query = {"where":"1=1",
             "outFields": "*",
             "outSR": '4326',         
             "returnGeometry": "true",
             "returnTrueCurves": "false",
             "returnIdsOnly": "false",
             "returnCountOnly": "false",
             "returnZ": "false",
             "returnM": "false",
             "orderByFields": "OBJECTID", 
             "returnDistinctValues": "false",
             "returnExtentsOnly": "false",
             "f":"json"}
    # First get request to get maxnumber and append first request
    r = requests.get(base_url, params = query, verify = False)
    return_json = r.json()
    features = return_json['features']
    record_max=(len(features))
    max_number = record_max
    rows = []
    # We first append the first set of features
    for feature in features:
        row = (feature['attributes']['CENTRELINE_ID'], 
                feature['attributes']['TRANS_ID_CREATE'],
                feature['attributes']['OBJECTID'],
                feature['attributes']['MI_PRINX'],
                feature['attributes']['REC_ID'],
                feature['attributes']['TYPE_CODE'],
                feature['attributes']['TYPE_DESC'],
                feature['attributes']['STATUS_CODE'],
                feature['attributes']['STATUS_DESC'],
                'SRID=4326;LineString('+','.join(' '.join(str(x) for x in tup) for tup in feature['geometry']['paths'][0]) +')')
            
        rows.append(row)
            
        # send to database    
    sql='INSERT INTO {} (geo_id, trans_id_create, object_id, mi_prinx, rec_id, type_code, type_desc, status_code, status_desc, geometry) VALUES %s'.format(output_table)
        
    with con:
        with con.cursor() as cur:
            execute_values(cur, sql, rows) 
            
    if return_json.get('exceededTransferLimit', False) == True:
        # Then loop the rest of the features
        while return_json.get('exceededTransferLimit', False) == True or check_status == 'error_raised':
                print('Current loop:', max_number, record_max)
                query = {"where": "1=1", 
                         "outFields": "*",
                         "outSR": '4326',         
                         "returnGeometry": "true",
                         "returnTrueCurves": "false",
                         "returnIdsOnly": "false",
                         "returnCountOnly": "false",
                         "returnZ": "false",
                         "returnM": "false",
                         "returnDistinctValues": "false",
                         "resultOffset": "{}".format(max_number),
                         "resultRecordCount": "{}".format(record_max),
                         "orderByFields": "OBJECTID", 
                         "returnExtentsOnly": "false",
                         "f":"json"}

                check_status1 = check_status(max_number, record_max)
                
                if check_status1 == 'successful':
                    r = requests.get(base_url, params = query, verify = False)
                    return_json = r.json()
                    features = return_json['features']
                    
                    print('length:', len(features))
                    rows = []
                    # We first append the first set of features
                    for feature in features:
                        row = (feature['attributes']['CENTRELINE_ID'], 
                               feature['attributes']['TRANS_ID_CREATE'],
                               feature['attributes']['OBJECTID'],
                               feature['attributes']['MI_PRINX'],
                               feature['attributes']['REC_ID'],
                               feature['attributes']['TYPE_CODE'],
                               feature['attributes']['TYPE_DESC'],
                               feature['attributes']['STATUS_CODE'],
                               feature['attributes']['STATUS_DESC'],
                               'SRID=4326;LineString('+','.join(' '.join(str(x) for x in tup) for tup in feature['geometry']['paths'][0]) +')')
                        rows.append(row)
                    # send to database    
                    sql='INSERT INTO {} (geo_id, trans_id_create, object_id, mi_prinx, rec_id, type_code, type_desc, status_code, status_desc, geometry) VALUES %s'.format(output_table)

                    with con:
                        with con.cursor() as cur:
                            execute_values(cur, sql, rows)       
                    print('inserted')
                    max_number = max_number + record_max
                    print('Next max_number: ', max_number)
                    
                else:
                    print('retrying')
                    continue
    else:
        print('all rows inserted')

In [301]:
get_layer('cot_geospatial2', 2, 'gcc_table')

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Current loop: 1000 1000


C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


length: 1000
inserted
Next max_number:  2000
Current loop: 2000 1000


C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


...

length: 1000
inserted
Next max_number:  6000
Current loop: 6000 1000


C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


length: 1000
inserted
Next max_number:  7000
Current loop: 7000 1000


C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


length: 49
inserted
Next max_number:  8000
